In [1]:
import os
os.getcwd()

'/home/jovyan/ArditArifi'

In [20]:
source_path="/home/jovyan/prepared-data/POLAR_EMISS_DATA/CMIP6"
output_path="/home/jovyan/student-storages/GROUP3/ArditArifi/output"

! mkdir -p output_path

Analysis

In [3]:
import glob
# ANALYSIS
import xarray as xr
import pandas as p
import numpy as np
# PLOTTING
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.colors as colors


In [21]:

# REGIONS
region_label = ["ARC", "ANT"]
arctic_lat = [60, 90]
antarctic_lat = [-90, -60]

# SEASONS
seasons_label = ["DJF", "MAM", "JJA", "SON"]
seasons = {
    "DJF": [12, 1, 2],   # Dec, Jan, Feb
    "MAM": [3, 4, 5],    # Mar, Apr, May
    "JJA": [6, 7, 8],    # Jun, Jul, Aug
    "SON": [9, 10, 11],  # Sep, Oct, Nov
}

# Create combined region-season labels
region_season_label = [f"{region} {season}" for region in region_label for season in seasons_label]

# EXPERIMENTS
experiments = ["control", "2xss", "2xdust", "2xfire", "2xDMS"]

# MODELS
models = ["IPSL-CM6A-LR-INCA", "UKESM1-0-LL", "NorESM2-LM"]

# PATTERN FUNCTION
patter = lambda model, var, exp: f"{source_path}/{var}*{model}*piClim-{exp}*.nc"

In [23]:
# CONSTRUCT TABLE
mean_values = np.zeros(shape=(len(region_season_label), len(experiments), len(models)))

# LOAD DATA
model = ""
var = "rtmt"

# MAIN LOOP
for idx_model, model in enumerate(models):
    for idx_exp, exp in enumerate(experiments):
        for idx_season, season in enumerate(seasons_label):

            # 1) Load dataset for this model/var/exp
            try:
                # Attempt to open the dataset
                ds = xr.open_mfdataset(patter(model, var, exp), combine="by_coords")
            except Exception as e:
                # If there's an error reading files, set to 0 for ARC and ANT
                print(f"Error opening files {patter(model, var, exp)}: {e}")
                mean_values[idx_season, idx_exp, idx_model] = 0
                mean_values[idx_season + 4, idx_exp, idx_model] = 0
                continue
                

            # 2) Select months for this season
            ds_season = ds_period.sel(time=ds_period.time.dt.month.isin(seasons[season]))

            # ---------------------
            # ARCTIC
            # ---------------------
            ds_season_arctic = ds_season.sel(
                lat=slice(arctic_lat[0], arctic_lat[1]),
                lon=slice(-180, 180)
            )
            weights_arctic = np.cos(np.deg2rad(ds_season_arctic.lat))

            # Weighted mean over lat/lon => time series
            ds_season_arctic_area_ts = ds_season_arctic[var].weighted(weights_arctic).mean(dim=("lat", "lon"))

            # Multi-year average (time)
            arc_value = ds_season_arctic_area_ts.mean("time").values
            # e.g. store in row idx_season for ARC
            mean_values[idx_season, idx_exp, idx_model] = arc_value

            # ---------------------
            # ANTARCTIC
            # ---------------------
            ds_season_antarctic = ds_season.sel(
                lat=slice(antarctic_lat[0], antarctic_lat[1]),
                lon=slice(-180, 180)
            )
            weights_antarctic = np.cos(np.deg2rad(ds_season_antarctic.lat))

            ds_season_antarctic_area_ts = ds_season_antarctic[var].weighted(weights_antarctic).mean(dim=("lat","lon"))
            ant_value = ds_season_antarctic_area_ts.mean("time").values
            # store in row idx_season+4 for ANT
            mean_values[idx_season + 4, idx_exp, idx_model] = ant_value

# Finally, display the populated array
print("Shape of mean_values:", mean_values.shape)
print(mean_values)

NameError: name 'ds_period' is not defined